<center>

# DATA COLLECTION

</center>

## Import libraries

In [1]:
import pandas as pd
import requests
import PyPDF2
import csv

## Import dataset

In [ ]:
url = "https://data.lacity.org/resource/2nrs-mtv8.csv?$select=*&$limit=764061&$offset=0"

# Descargar el csv
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 2xx)
if str(response.status_code).startswith('2'):
    # Guardar el contenido en un archivo CSV
    with open("../data/raw/crime_data.csv", "wb") as f:
        f.write(response.content)
else:
    print(f"Error al realizar la solicitud: {response.status_code}")

In [15]:
url_pdf = "https://data.lacity.org/api/views/2nrs-mtv8/files/4591b6bf-5846-4de0-9fb0-8780a77a036c?download=true&filename=MO_CODES_Numerical_20191119.pdf"

# Download the PDF file
response = requests.get(url_pdf)

# Check if the request was successful (2xx status code)
if str(response.status_code).startswith('2'):
    # Save the content to a local file
    with open("../data/raw/mo_codes_data.pdf", "wb") as pdf_file:
        pdf_file.write(response.content)

    # Open the file locally and extract the text
    with open("../data/raw/mo_codes_data.pdf", "rb") as local_pdf:
        pdf_reader = PyPDF2.PdfReader(local_pdf)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
            
    # Remove the word "MO CODES"
    text = text.replace("MO CODES", "")

    # Save the text as a CSV file
    with open("../data/raw/mo_codes_data.csv", "w", encoding="utf-8", newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        # Split the text into lines and write each line as a row in the CSV
        for line in text.split('\n'):
            csv_writer.writerow([line.strip()])

else:
    print(f"Error al realizar la solicitud: {response.status_code}")